#### 감성 분석 모델 작업 및 추론

#### 1. 데이터 로드

In [43]:
import pandas as pd
import numpy as np

In [44]:
df_train = pd.read_excel('data/training/감성대화말뭉치(최종데이터)_Training.xlsx')
df_test = pd.read_excel('data/validation/감성대화말뭉치(최종데이터)_Validation.xlsx')

In [45]:
df_train = df_train[['감정_대분류', '사람문장1']].rename(columns = {'감정_대분류': '감정','사람문장1':'문장'})
df_test = df_test[['감정_대분류', '사람문장1']].rename(columns = {'감정_대분류': '감정','사람문장1':'문장'})

#### 2. 데이터 전처리

In [46]:
# 결측치 처리
print(df_train.isnull().sum())
print(df_test.isnull().sum())

감정    0
문장    0
dtype: int64
감정    0
문장    0
dtype: int64


In [47]:
import re
import torch
import torch.nn.functional as F

def load_stopwords(filepath):
    with open(filepath, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords

def preprocess(sentence, okt):

    # 숫자, 영문, 특수문자, 이모지, 공백 제거
    sentence = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', sentence)

    # 어근 분리 및 토큰화
    tokens = okt.morphs(sentence, stem =True)

    # 불용어 제거
    ko_stopwords = load_stopwords('data/ko_stopwords.txt')
    cleaned_tokens = [token for token in tokens if token not in ko_stopwords]

    return cleaned_tokens

def pad_sequences(sequences, maxlen, padding_value=0):
    padded_sequences = [F.pad(seq[:maxlen], (0, max(0, maxlen-len(seq))), value=padding_value) for seq in sequences]
    return torch.stack(padded_sequences)

In [48]:
from konlpy.tag import Okt
# 진행률 바를 표시
from tqdm import tqdm

okt = Okt()
vocab = {}
train_data = []
test_data = []

for sentence in tqdm(df_train['문장']):
    tokens = preprocess(sentence, okt)

    for token in tokens:
        if token not in vocab:
            vocab[token] = 1
        else:
            vocab[token] += 1

    train_data.append(tokens)

for sentence in tqdm(df_test['문장']):
    tokens = preprocess(sentence, okt)
    test_data.append(tokens)

100%|██████████| 6641/6641 [00:12<00:00, 542.40it/s]


In [49]:
train_data[:10], vocab

([['은', '해도', '해도', '끝', '없다', '화가', '나다'],
  ['달', '급여', '깎다', '물가', '는', '월급', '만', '자꾸', '깎다', '너무', '화가'],
  ['회사',
   '신입',
   '들어오다',
   '말투',
   '거슬리다',
   '그렇다',
   '애',
   '매일',
   '보다',
   '하다',
   '생각',
   '하다',
   '스트레스',
   '받다'],
  ['직장',
   '막내',
   '라는',
   '이유',
   '에게만',
   '온갖',
   '심부름',
   '일도',
   '많다',
   '데',
   '정말',
   '분하다',
   '섭섭하다'],
  ['전', '입사', '한', '신입사원', '나르다', '무시', '하다', '너무', '화가'],
  ['직장', '다니다', '만', '버리다', '거', '진지하다', '진로', '대한', '고민', '생기다'],
  ['성인',
   '인데',
   '도',
   '진로',
   '아직도',
   '못',
   '정',
   '하다',
   '부모님',
   '노',
   '워',
   '하다',
   '나다',
   '섭섭하다'],
  ['퇴사', '한', '지다', '안', '돼다', '천천히', '직장', '해보다'],
  ['졸업',
   '반',
   '이라서',
   '취업',
   '생각',
   '하다',
   '하다',
   '지금',
   '너무',
   '느긋하다',
   '이래도',
   '되다',
   '싶다'],
  ['요즘', '직장', '생활', '너무', '편하다', '좋다']],
 {'은': 3861,
  '해도': 300,
  '끝': 84,
  '없다': 5630,
  '화가': 1308,
  '나다': 1686,
  '달': 295,
  '급여': 38,
  '깎다': 25,
  '물가': 10,
  '는': 5526,
  '월급': 213,
  '만': 3042,
 

In [80]:
vocab_size = 10000
vocab_sorted = sorted(vocab.items(), key=lambda item: item[1], reverse=True)

word_to_idx = {word: i+1 for i, (word, _) in enumerate(vocab_sorted)}
word_to_idx = {word: index for word, index in word_to_idx.items() if index <= vocab_size}
word_to_idx['OOV'] = len(word_to_idx) + 1

In [81]:
oov_idx = word_to_idx['OOV']
enc_train = []
enc_test = []

enc_train = [[word_to_idx.get(token, oov_idx) for token in sentence] for sentence in tqdm(train_data)]
enc_test = [[word_to_idx.get(token, oov_idx) for token in sentence] for sentence in tqdm(test_data)]


100%|██████████| 6641/6641 [00:00<00:00, 423042.28it/s]


In [132]:
train_seq = [torch.tensor(seq) for seq in enc_train]
test_seq = [torch.tensor(seq) for seq in enc_test]

In [133]:
import torch.nn as nn

EMBEDDING_DIM = 64

num_words = len(word_to_idx)+1
SEQ_LEN = max([len(sentence) for sentence in enc_train])

train_input = pad_sequences(train_seq, maxlen=SEQ_LEN)
test_input = pad_sequences(test_seq, maxlen=SEQ_LEN)
print(len(train_input[0]))

47


In [134]:
labels = df_train['감정'].unique()

enc_labels = {label : i for i, label in enumerate(labels)}

train_targets = [enc_labels[label] for label in df_train['감정']]
test_targets = [enc_labels[label] for label in df_test['감정']]

train_targets = torch.tensor(train_targets)
test_targets = torch.tensor(test_targets)

num_classes = len(labels)

#### 3. 모델 정의 및 생성

In [135]:
class EBDSentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.3):
        super(EBDSentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        out = self.fc(hidden[-1])
        return out

In [136]:
HIDDEN_DIM = 64
OUTPUT_DIM = num_classes  
EMBEDDING_DIM = 128

model = EBDSentimentLSTM(
    vocab_size=num_words,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM
)

print(model)

EBDSentimentLSTM(
  (embedding): Embedding(10002, 128)
  (lstm): LSTM(128, 64, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=64, out_features=6, bias=True)
)


c:\Users\Playdata\AppData\Local\anaconda3\envs\nlp1_env\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


#### 4. 모델 학습

##### RNN 
- 순서가 있는 데이터 (문장, 음성, 시계열)을 다루기 위해 고안된 신경망
- 핵심 아이디어 : 이전까지의 정보를 Hidden state에 저장해 다음 단계로 넘어가며 한 토큰씩 처리

**비유**
- 문장을 읽을 때 우리 뇌는 앞에서 본 내용(기억)을 머리속에 두고 다음 단어를 이해한다
- RNN도 똑같이, 매 시점에 입력 + 이전 기억 (은닉 상태) -> 새 기억을 만들며 한칸 씩 진행한다

**필요성**
- 문장 / 음성 처럼 순서와 맥락이 중요한 데이터는 앞 내용이 현재 해석에 영향을 준다
- RNN은 이 앞 내용을 h에 저장해 다음 단계로 전달하므로 시간적 의존성을 배운다 

In [137]:
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset,random_split

# 배치 사이즈, 학습/검증셋 크기 설정
BATCH_SIZE = 65
train_size = int(len(train_input) * 0.8)
val_size = len(train_input) - train_size

# label 데이터 실수 처리
train_target = train_targets
test_target = test_targets

# 학습/검증셋 분할
train_dataset, val_dataset = random_split(TensorDataset(train_input, train_target),[train_size, val_size])

# 미니배치로 사용할 수 있도록 DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE)

# epoch (학습 횟수), 손실함수, 최적화 함수 정의
epochs = 100
criterion =  nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.005)

# 시각화를 위한 손실값 / 정확도 저장용 배열 생성
train_losses, val_losses, train_accs, val_accs = [], [], [], []

# 조기종료 관련 변수 초기화
early_stopping_patience = 7
best_val_loss = float('inf')
early_stop_counter = 0

In [114]:
print(train_input.shape, train_target.shape)

torch.Size([51630, 47]) torch.Size([51630])


In [102]:
print(train_target)

tensor([0., 0., 0.,  ..., 5., 2., 5.])


In [144]:
for epoch in range(epochs):

    # 학습 모드
    model.train()
    total_loss, correct, total = 0, 0, 0

    for inputs, targets in train_loader:
        optimizer.zero_grad()                   # 가중치 초기화
        # model(inptus) : 모델의 순전파를 돌려 예측 텐서 (보통 로짓 / 확률)를 얻고
        # .squeeze() : 그 텐서에서 크기가 1인 차원을 모두 제거
        # e.g. [1,1,256] - >[256]
        outputs = model(inputs)       # 순전파 
        loss = criterion(outputs, targets)      # 손실 계산
    

        loss.backward()                         # 역전파
        optimizer.step()                        # 가중치 갱신
        total_loss += loss.item()

        pred = outputs.argmax(dim=1)
        correct += (pred == targets).sum().item()
        total += targets.size(0)

    train_loss = total_loss / len(train_loader)
    train_acc = correct / total

    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # 검증 모드
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0

    with torch.no_grad(): 
        for val_inputs, val_targets in val_loader:
            val_outputs = model(val_inputs)


            loss = criterion(val_outputs, val_targets)      # 손실 계산
            val_loss += loss.item()


            val_pred = (val_outputs > 0.5).float()
            val_correct += (val_pred == val_targets).sum().item()
            val_total += val_targets.size(0)

    val_loss = val_loss / len(val_loader)
    val_acc = val_correct / val_total

    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f'Epoch {epoch + 1}/{epochs} | Train Loss: {train_loss: .4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0

    else:
        early_stop_counter += 1
        if early_stop_counter >= early_stopping_patience:
            print("Early Stopping 할게!")
            break

KeyboardInterrupt: 

#### 5. 추론

In [145]:
def predict_sentiment(sentence, word_to_idx, okt):
    tokens = preprocess(sentence, okt)
    oov_idx = word_to_idx.get('OOV', 0)
    encoded = [word_to_idx.get(token, oov_idx) for token in tokens]
    sequnce = torch.tensor(encoded, dtype=torch.long).unsqueeze(0)
    input = pad_sequences(sequnce, maxlen=SEQ_LEN)

    with torch.no_grad():
        output = model(input)
        pred = output.argmax(dim=1).item()
    return pred

In [ ]:
sentence = "너무 기뻐"
pred_label = predict_sentiment(sentence)
print("예측 라벨:", pred_label)